<h1><center>Pitroski F Score</center></h1>

## 1. Data Extraction

I have used requests and BeautifulSoup library to carry out web scraping.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

Pitroski's F Score works for Small and Mid Cap stocks. I have selected a few stocks out of S&P 400 MidCap.

In [2]:
tickers = ["AMCX","ARW","AN","ATO","AVT","CZR","CABO","CFX", "CNX",
           "DPZ","FNB","ITT","MAN","MCY","SIX","X","Y","WPX","AAPL",
          "DEI","DECK","DLX","EGP","EQT","WTRG","EXP","FFIN","FLO","MSFT"
           ,"FL","GEO","HQY","JEF","MMS","ORI","PEB","PENN","PZZA","SABR"]

I have extracted data from Yahoo Finance using web scraping.

In [3]:
financial_dir_curr = {} 
financial_dir_py = {} 
financial_dir_py2 = {} 

In [4]:
for ticker in tickers:
    try:
        print("scraping financial statement data for ",ticker)
        temp_dir = {}
        temp_dir2 = {}
        temp_dir3 = {}
    #getting balance sheet data from yahoo finance 
        url = 'https://in.finance.yahoo.com/quote/'+ticker+'/balance-sheet?p='+ticker
#         print(url)
        page = requests.get(url)
        page_content = page.content
        soup = BeautifulSoup(page_content,'html.parser')
        tabl = soup.find_all("div", {"class" : "M(0) Whs(n) BdEnd Bdc($seperatorColor) D(itb)"})
        for t in tabl:
            rows = t.find_all("div", {"class" : "rw-expnded"})
            for row in rows:
                temp_dir[row.get_text(separator='|').split("|")[0]]=row.get_text(separator='|').split("|")[1]
                temp_dir2[row.get_text(separator='|').split("|")[0]]=row.get_text(separator='|').split("|")[2]
                temp_dir3[row.get_text(separator='|').split("|")[0]]=row.get_text(separator='|').split("|")[3]
        
        #getting income statement data from yahoo finance 
        url = 'https://in.finance.yahoo.com/quote/'+ticker+'/financials?p='+ticker
#         print(url)
        page = requests.get(url)
        page_content = page.content
        soup = BeautifulSoup(page_content,'html.parser')
        tabl = soup.find_all("div", {"class" : "M(0) Whs(n) BdEnd Bdc($seperatorColor) D(itb)"})
        for t in tabl:
            rows = t.find_all("div", {"class" : "rw-expnded"})
            for row in rows:
                temp_dir[row.get_text(separator='|').split("|")[0]]=row.get_text(separator='|').split("|")[1]
                temp_dir2[row.get_text(separator='|').split("|")[0]]=row.get_text(separator='|').split("|")[2]
                temp_dir3[row.get_text(separator='|').split("|")[0]]=row.get_text(separator='|').split("|")[3]

        
        #getting cashflow statement data from yahoo finance 
        url = 'https://in.finance.yahoo.com/quote/'+ticker+'/cash-flow?p='+ticker
        page = requests.get(url)
#         print(url)
        page_content = page.content
        soup = BeautifulSoup(page_content,'html.parser')
        tabl = soup.find_all("div", {"class" : "M(0) Whs(n) BdEnd Bdc($seperatorColor) D(itb)"})
        for t in tabl:
            rows = t.find_all("div", {"class" : "rw-expnded"})
            for row in rows:
                temp_dir[row.get_text(separator='|').split("|")[0]]=row.get_text(separator='|').split("|")[1]
                temp_dir2[row.get_text(separator='|').split("|")[0]]=row.get_text(separator='|').split("|")[2]
                temp_dir3[row.get_text(separator='|').split("|")[0]]=row.get_text(separator='|').split("|")[3] 
        
        financial_dir_curr[ticker] = temp_dir
        financial_dir_py[ticker] = temp_dir2
        financial_dir_py2[ticker] = temp_dir3
    except:
        print("Problem scraping data for ",ticker)

scraping financial statement data for  AMCX
scraping financial statement data for  ARW
scraping financial statement data for  AN
scraping financial statement data for  ATO
scraping financial statement data for  AVT
scraping financial statement data for  CZR
scraping financial statement data for  CABO
scraping financial statement data for  CFX
scraping financial statement data for  CNX
scraping financial statement data for  DPZ
scraping financial statement data for  FNB
scraping financial statement data for  ITT
scraping financial statement data for  MAN
scraping financial statement data for  MCY
scraping financial statement data for  SIX
scraping financial statement data for  X
scraping financial statement data for  Y
scraping financial statement data for  WPX
scraping financial statement data for  AAPL
scraping financial statement data for  DEI
scraping financial statement data for  DECK
scraping financial statement data for  DLX
scraping financial statement data for  EGP
scraping fin

I had saved the data in directory. Below, I have saved the directory into Pandas Dataframe. <br>
There is also a possibility that data of few tickers might not have been imported and hence I am updating the tickers list.

In [5]:
combined_financials_curr = pd.DataFrame(financial_dir_curr)
combined_financials_py = pd.DataFrame(financial_dir_py)
combined_financials_py2 = pd.DataFrame(financial_dir_py2)
tickers = combined_financials_curr.columns 
index = combined_financials_curr.index

## 2. Data Cleaning and Calculation

I had extracted a lot of data, but we only need a few specific details and hence I have compiled them in a dataframe below.

In [6]:
stats = ["Net income available to common shareholders",
         "Total assets",
         "Net cash provided by operating activities",
         "Long-term debt",
         "Other long-term liabilities",
         "Total current assets",
         "Total current liabilities",
         "Common stock",
         "Total revenue",
         "Gross profit"] 
indx = ["NetIncome","TotAssets","CashFlowOps","LTDebt","OtherLTDebt",
        "CurrAssets","CurrLiab","CommStock","TotRevenue","GrossProfit"]

There is a possibilty that the name in the target dataframe might not match the extracted dataframe and hence I am checking for that below.

In [7]:
for i in stats:
    if i not in index:
        print( i + ": Error, check row name")

In [8]:
all_stats = {}
for ticker in tickers:
    try:
#         print(ticker)
        temp = combined_financials_curr[ticker]
#         print(temp)
        ticker_stats = []
        for stat in stats:
#             print(temp.loc[stat])
            ticker_stats.append(temp.loc[stat])
        all_stats['{}'.format(ticker)] = ticker_stats
        
    except:
        print("can't read data for ",ticker)
df_curr = pd.DataFrame(all_stats,index=indx)
df_curr

,AMCX,ARW,AN,ATO,AVT,CZR,CABO,CFX,CNX,DPZ,...,FL,GEO,HQY,JEF,MMS,ORI,PEB,PENN,PZZA,SABR
NetIncome,"3,05,756","-2,95,319","1,25,700","5,32,016","-1,15,078","-7,89,000","2,09,169","-4,71,142","-3,22,479","4,29,661",...,"2,09,000","1,51,079",-332,"4,00,612","2,09,371","39,400","1,18,929","-6,05,700","-15,570","-1,10,938"
TotAssets,"55,96,686","1,64,00,796","1,05,43,300","1,33,67,619","85,64,556","2,53,45,000","31,51,831","73,86,832","90,60,806","13,82,092",...,"65,89,000","43,17,534","25,64,981","4,94,60,234","17,45,732","2,10,76,300","64,98,555","1,41,94,500","7,30,721","56,89,957"
CashFlowOps,"5,10,469","16,53,859","6,23,200","9,76,530","7,77,235","7,32,000","5,05,863","2,59,465","9,39,295","4,95,310",...,"2,62,000","3,67,391","94,700",-,"3,38,788","10,16,300","3,40,356","5,45,000","17,762","4,66,359"
LTDebt,"30,39,979","26,40,129","15,78,500","35,29,452","14,19,922","84,78,000","17,11,937","22,84,184","27,54,443","40,52,792",...,"1,22,000","27,17,533","11,81,615","83,37,061",-,NaN,"22,29,220","23,22,200","3,47,290","32,61,821"
OtherLTDebt,"4,03,451","6,36,115","2,62,400","74,144","4,25,585",-,"1,05,469","6,19,307","42,320",NaN,...,"9,000","88,526","2,625",NaN,"20,313",NaN,NaN,"98,000","6,556","95,069"
CurrAssets,"23,30,297","1,25,26,159","44,11,100","4,58,031","68,76,336","26,09,000","1,81,488","14,04,245","4,98,101","7,87,617",...,"23,86,000","5,47,778","2,97,300","4,22,52,408","7,89,425",NaN,"1,65,968","6,42,800","1,81,546","11,21,920"
CurrLiab,"8,04,342","82,58,159","51,00,200","12,09,440","25,78,576","20,63,000","1,89,542","8,57,314","5,28,939","4,53,831",...,"11,94,000","3,95,928","1,51,937","3,13,69,884","3,64,238",NaN,"5,79,695","9,05,600","2,07,945","10,25,543"
CommStock,754,"1,25,424","1,000",597,"1,04,038","7,000",59,118,"1,870",389,...,"7,64,000","1,254",7,"4,16,644","4,98,433","3,03,600","1,509","1,200",447,"2,943"
TotRevenue,"30,10,475","2,81,42,273","2,10,20,900","28,99,629","1,81,55,541","84,55,000","12,10,588","34,59,895","13,58,315","36,55,913",...,"71,03,000","24,72,272","6,34,952","44,96,793","31,22,040","60,46,500","15,14,151","51,34,900","15,09,745","35,84,604"
GrossProfit,"14,99,578","31,65,084","34,87,000","20,86,089","21,83,510","41,75,000",-,"14,88,257","3,28,614","14,20,367",...,"21,25,000","6,07,189","3,76,230","16,68,595","6,71,045",NaN,"4,51,994","22,31,700","2,71,643","7,25,649"


In [9]:
all_stats = {}
for ticker in tickers:
    try:
#         print(ticker)
        temp = combined_financials_py[ticker]
#         print(temp)
        ticker_stats = []
        for stat in stats:
#             print(temp.loc[stat])
            ticker_stats.append(temp.loc[stat])
        all_stats['{}'.format(ticker)] = ticker_stats
    except:
        print("can't read data for ",ticker)
df_py = pd.DataFrame(all_stats,index=indx)
df_py

,AMCX,ARW,AN,ATO,AVT,CZR,CABO,CFX,CNX,DPZ,...,FL,GEO,HQY,JEF,MMS,ORI,PEB,PENN,PZZA,SABR
NetIncome,"3,80,486","-2,04,087","4,50,000","5,10,990","1,76,337","-11,95,000","1,78,582","-5,27,646","-80,730","4,00,709",...,"4,91,000","1,66,603","39,664","9,54,017","2,40,824","10,56,400","82,592","43,900","-7,633","1,58,592"
TotAssets,"52,78,563","1,77,84,445","1,06,65,100","1,18,74,437","95,96,845","2,57,75,000","23,03,234","66,03,872","85,92,170","9,07,385",...,"38,20,000","42,47,463","5,10,016","4,71,69,108","14,62,000","1,93,27,100","69,78,348","1,09,61,012","5,70,947","58,06,381"
CashFlowOps,"4,83,748","8,57,995","7,69,200","9,68,769","5,34,770","10,07,000","4,91,741","1,30,948","9,80,560","4,96,950",...,"6,96,000","3,38,143","1,05,010","-7,17,181","3,56,727","9,36,200","3,95,202","7,03,900","61,749","5,78,877"
LTDebt,"30,88,221","32,39,115","19,26,200","24,93,665","14,89,219","88,01,000","11,42,056","11,92,408","23,78,205","34,95,691",...,"1,24,000","24,12,244",-,"78,85,783",-,NaN,"27,46,898","23,50,088","6,01,126","33,37,467"
OtherLTDebt,"5,81,285","3,78,536","2,75,000","55,429","4,53,084",-,"9,980","7,35,173","1,59,787",NaN,...,"2,74,000","82,481","2,968",NaN,"17,864",NaN,NaN,"28,269","2,923","1,45,891"
CurrAssets,"19,63,411","1,36,08,300","48,84,000","4,78,853","76,10,599","22,65,000","3,17,863","19,58,441","5,01,686","5,66,951",...,"25,18,000","5,91,107","3,94,677","3,83,35,148","8,08,632",NaN,"2,44,410","6,77,658","1,71,708","11,87,630"
CurrLiab,"7,97,421","87,90,428","56,58,100","19,15,131","29,69,461","20,00,000","1,33,713","11,99,345","5,22,975","3,79,743",...,"7,64,000","6,94,583","29,053","2,85,92,753","2,77,449",NaN,"4,06,779","7,38,436","1,64,636","10,18,395"
CommStock,748,"1,25,424","1,000",556,"1,15,825","7,000",59,117,"1,990",410,...,"8,09,000","1,248",6,"4,81,227","4,87,539","3,02,700","1,507","1,188",443,"2,917"
TotRevenue,"30,60,321","2,89,16,847","2,13,35,700","29,01,848","1,95,18,592","87,42,000","11,67,997","33,27,458","15,32,666","36,18,774",...,"80,05,000","24,77,922","5,31,993","38,92,976","28,86,815","72,13,600","16,12,213","53,01,400","16,19,248","39,74,988"
GrossProfit,"15,53,336","32,98,381","35,23,000","20,43,011","24,86,102","44,20,000","7,79,445","14,01,056","5,10,207","14,02,499",...,"25,43,000","6,17,164","3,25,909","14,38,127","6,71,184",NaN,"5,13,032","23,46,900","5,23,831","9,39,985"


In [10]:
all_stats = {}
for ticker in tickers:
    try:
#         print(ticker)
        temp = combined_financials_py2[ticker]
#         print(temp)
        ticker_stats = []
        for stat in stats:
#             print(temp.loc[stat])
            ticker_stats.append(temp.loc[stat])
        all_stats['{}'.format(ticker)] = ticker_stats
    except:
        print("can't read data for ",ticker)
df_py2 = pd.DataFrame(all_stats,index=indx)
df_py2

,AMCX,ARW,AN,ATO,AVT,CZR,CABO,CFX,CNX,DPZ,...,FL,GEO,HQY,JEF,MMS,ORI,PEB,PENN,PZZA,SABR
NetIncome,"4,46,187","7,16,195","3,96,000","6,02,484","-1,56,424","3,03,000","1,64,760","1,40,196","7,96,533","3,61,972",...,"5,41,000","1,45,089","73,899","1,58,761","2,20,751","3,70,500","-4,405","93,519","1,646","3,37,531"
TotAssets,"50,32,985","1,64,62,809","1,02,71,500","1,07,49,596","96,99,589","2,55,12,000","22,18,329","67,09,697","69,31,913","8,36,753",...,"39,61,000","42,26,908","3,69,159","4,50,71,307","13,50,662","1,94,03,500","25,90,868","52,34,812","5,55,553","56,49,364"
CashFlowOps,"6,06,547","2,72,690","5,11,000","11,24,662","2,53,485","7,86,000","4,07,769","2,26,367","8,85,823","3,94,171",...,"7,81,000","2,74,475","1,13,422",-,"3,23,525","7,60,500","1,35,697","3,52,794","72,795","7,22,902"
LTDebt,"30,99,257","29,33,045","19,59,200","30,67,045","17,29,212","88,49,000","11,60,682","10,55,305","21,87,026","31,21,490",...,"1,25,000","25,46,908",-,"73,80,443",527,NaN,"8,85,237","12,14,625","4,46,565","33,98,731"
OtherLTDebt,"6,71,102","5,72,971","2,35,400","36,266","3,34,538","20,50,000",-,"8,29,748","1,05,073",NaN,...,"2,54,000","96,702","2,395",NaN,"9,106",NaN,NaN,"46,652","2,332","2,66,027"
CurrAssets,"18,79,850","1,24,17,759","47,97,500","5,39,646","75,33,808","34,59,000","2,42,384","20,69,832","8,52,504","5,79,780",...,"25,51,000","5,79,709","2,65,396","3,55,63,090","6,57,242",NaN,"1,05,381","4,01,032","1,69,994","9,60,692"
CurrLiab,"7,44,102","79,56,569","56,35,700","10,13,443","24,53,771","18,88,000","1,70,604","10,97,380","4,41,679","3,98,285",...,"6,16,000","3,69,563","20,490","2,69,25,406","3,16,266",NaN,"1,82,717","5,30,009","1,32,860","9,72,147"
CommStock,742,"1,25,424","1,000",531,"1,23,081","7,000",60,123,"2,241",429,...,"8,42,000","1,240",6,"4,84,425","4,75,592","2,69,200",788,933,442,"2,891"
TotRevenue,"29,71,929","2,96,76,768","2,14,12,800","31,15,546","1,90,36,892","83,91,000","10,72,295","36,66,812","17,33,704","34,32,867",...,"79,39,000","23,31,386","2,87,243","31,83,376","23,92,236","60,21,700","8,28,678","35,87,918","15,73,316","38,66,956"
GrossProfit,"15,25,980","37,00,912","33,97,300","19,47,698","25,27,184","42,10,000","7,02,026","11,32,839","7,44,642","13,02,679",...,"25,28,000","5,75,614","1,81,193","12,58,044","5,94,385",NaN,"2,62,653","15,39,954","4,21,414","10,75,542"


In [11]:
df_curr[tickers] = df_curr[tickers].replace({',': ''}, regex=True)
for ticker in df_curr.columns:
    df_curr[ticker] = pd.to_numeric(df_curr[ticker].values,errors='coerce')

In [12]:
df_py[tickers] = df_py[tickers].replace({',': ''}, regex=True)
for ticker in df_py.columns:
    df_py[ticker] = pd.to_numeric(df_py[ticker].values,errors='coerce')

In [13]:
df_py2[tickers] = df_py2[tickers].replace({',': ''}, regex=True)
for ticker in df_py2.columns:
    df_py2[ticker] = pd.to_numeric(df_py2[ticker].values,errors='coerce')

## 3. Pitroski F Score Calculation

Pitroski's F score is used to access the companies financial position and is rated from 0 to 9.<br>
Score is calculated based on 3 criterions:
1. Profitability Index
    - (+) Return on on assets in the current year(1 point)
    - (+) Operating Cash Flow in the current year(1 point)
    - ROA is higher in current year compared to the previous year(1 point)
    - CF/Total Assets > ROA in current year(1 point)
2. Leverage, Liquidity and Source of Funds
    - Lower Ratio of long Term Debt in current year compared to last year, decreased leverage(1 point)
    - Higher current ratio in current year compared to last year, more liquidity(1 point)
    - No new shares were issued this year, lack of dilution(1 point)
3. Operating Efficiency Criterion
    - Higher Gross Margin compared in current year compared to last year(1 point)
    - Higher Asset Turnover ratio compared in current year compared to last year(1 point)


In [14]:
f_score = {}
tickers = df_curr.columns
for ticker in tickers:
    ROA_FS = int(df_curr.loc["NetIncome",ticker]/((df_curr.loc["TotAssets",ticker]+df_py.loc["TotAssets",ticker])/2) > 0)
    CFO_FS = int(df_curr.loc["CashFlowOps",ticker] > 0)
    ROA_D_FS = int(df_curr.loc["NetIncome",ticker]/(df_curr.loc["TotAssets",ticker]+df_py.loc["TotAssets",ticker])/2 > df_py.loc["NetIncome",ticker]/(df_py.loc["TotAssets",ticker]+df_py2.loc["TotAssets",ticker])/2)
    CFO_ROA_FS = int(df_curr.loc["CashFlowOps",ticker]/df_curr.loc["TotAssets",ticker] > df_curr.loc["NetIncome",ticker]/((df_curr.loc["TotAssets",ticker]+df_py.loc["TotAssets",ticker])/2))
    LTD_FS = int((df_curr.loc["LTDebt",ticker] + df_curr.loc["OtherLTDebt",ticker])<(df_py.loc["LTDebt",ticker] + df_py.loc["OtherLTDebt",ticker]))
    CR_FS = int((df_curr.loc["CurrAssets",ticker]/df_curr.loc["CurrLiab",ticker])>(df_py.loc["CurrAssets",ticker]/df_py.loc["CurrLiab",ticker]))
    DILUTION_FS = int(df_curr.loc["CommStock",ticker] <= df_py.loc["CommStock",ticker])
    GM_FS = int((df_curr.loc["GrossProfit",ticker]/df_curr.loc["TotRevenue",ticker])>(df_py.loc["GrossProfit",ticker]/df_py.loc["TotRevenue",ticker]))
    ATO_FS = int(df_curr.loc["TotRevenue",ticker]/((df_curr.loc["TotAssets",ticker]+df_py.loc["TotAssets",ticker])/2)>df_py.loc["TotRevenue",ticker]/((df_py.loc["TotAssets",ticker]+df_py2.loc["TotAssets",ticker])/2))
    f_score[ticker] = [ROA_FS,CFO_FS,ROA_D_FS,CFO_ROA_FS,LTD_FS,CR_FS,DILUTION_FS,GM_FS,ATO_FS]
    f_score_df = pd.DataFrame(f_score,index=["PosROA","PosCFO","ROAChange","Accruals","Leverage","Liquidity","Dilution","GM","ATO"])

C:\Users\chashah4\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in less
  


In [15]:
f_score_df

,AMCX,ARW,AN,ATO,AVT,CZR,CABO,CFX,CNX,DPZ,...,FL,GEO,HQY,JEF,MMS,ORI,PEB,PENN,PZZA,SABR
PosROA,1,0,1,1,0,0,1,0,0,1,...,1,1,0,1,1,1,1,0,0,0
PosCFO,1,1,1,1,1,1,1,1,1,1,...,1,1,1,0,1,1,1,1,1,1
ROAChange,0,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
Accruals,1,1,1,1,1,1,1,1,1,0,...,0,1,1,0,1,1,1,1,1,1
Leverage,1,1,1,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,1
Liquidity,1,0,1,1,1,1,0,1,0,1,...,0,1,0,1,0,0,0,0,0,0
Dilution,0,1,1,0,1,1,1,0,1,1,...,1,0,0,1,0,0,0,0,0,0
GM,0,0,1,1,0,0,0,1,0,1,...,0,0,0,1,0,0,0,0,0,0
ATO,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [16]:
f_score_df.sum().sort_values(ascending=False)

MSFT    7
AN      7
AAPL    7
DLX     6
EQT     6
FLO     6
DPZ     5
EXP     5
SIX     5
MAN     5
EGP     5
ITT     5
AMCX    5
ATO     5
CFX     5
CZR     5
JEF     5
AVT     5
Y       4
CABO    4
ARW     4
DEI     4
WPX     4
FL      4
PEB     4
DECK    4
WTRG    4
GEO     4
ORI     3
MMS     3
CNX     3
FNB     3
MCY     3
PZZA    3
SABR    3
X       2
FFIN    2
HQY     2
PENN    2
dtype: int64